In [16]:
#HIDE
try:
    import setup
except:
    pass
from notebook_helper import *
notebook_setup()
from modules import la

# Index of Multiple deprivation approach

This creates a basic score for each local authority based on the overall deprivation score for an authority. 

This uses a GB wide approach as Northern Ireland councils do not overlap in the same table. It calculates population quintiles, so the number is not exactly even. 

This is based on the [composite 2020 IMD scoring](https://github.com/mysociety/composite_uk_imd), in turn based on each national IMD ranking.

In this end this generated two possible sets of groupings. One where all lower-tier authorities were combined into a single ranking, and another where both leagues have seperate groupings. 

In [17]:

df = pd.read_csv(Path("data", "source", "GB_IMD_E.csv")).set_index("lsoa")
pop = pd.read_csv(Path("data", "source", "2019_population.csv"),
                  thousands=",").set_index("lsoa")
lookup = pd.read_csv(
    Path("data", "source", "lsoa_la_2021.csv")).set_index("lsoa")

df = df.join(pop, how="outer").join(lookup, how="outer")
df = df[~df["nation"].isna()] # exclude NI

# create a population adjusted score
df["pop_score"] = df["GB_IMD_E_score"] * df["pop"]

# pivot up to the local authority level
df = df.pivot_table(["pop_score", "pop"],
                    index="local-authority-code", aggfunc="sum")


def calculate_deciles(df):

    # calculate a new score, dividing the summed score by the summed population
    df["la_deprivation_score"] = df["pop_score"] / df["pop"]
    df = df.sort_values("la_deprivation_score", ascending=False)

    df["cum_pop"] = df["pop"].astype("int").cumsum()

    df["la_imd_pop_decile"] = np.ceil(
        df["cum_pop"]/sum(df["pop"]) * 10).astype(int)
    df["la_imd_pop_quintile"] = np.ceil(
        df["cum_pop"]/sum(df["pop"]) * 5).astype(int)

    df = df.drop(columns=["cum_pop"])
    return df

df = calculate_deciles(df)

#df = df.drop(columns=["pop", "pop_score"])

(df
 .reset_index()
 .pivot_table("local-authority-code", columns="la_imd_pop_quintile", aggfunc="count")
 .fillna(0)
 .astype(int)
 .common.row_percentages()
 .style.format(percentage_0dp))


la_imd_pop_quintile,1,2,3,4,5
local-authority-code,15%,14%,22%,21%,27%


The number of councils is not even because more deprived councils are also more populus, so more of the population living in a deprived area lives in a smaller set of councils. 

### Distribution as row percentages

In [18]:
ldf = la.get_la_with_leagues().set_index("local-authority-code")
ldf = ldf.join(df, how="outer")

(ldf
 .reset_index()
 .pivot_table("local-authority-code", index="league-group", columns="la_imd_pop_quintile", aggfunc="count")
 .fillna(0)
 .astype(int)
 .common.row_percentages()
 .style.format(percentage_0dp))


la_imd_pop_quintile,1.0,2.0,3.0,4.0,5.0
league-group,,,,,
District councils,6%,6%,22%,25%,42%
Single tier,24%,23%,23%,18%,12%


### Distribution in number of councils

In [19]:
(ldf
 .reset_index()
 .pivot_table("local-authority-code", index="league-group", columns="la_imd_pop_quintile", aggfunc="count")
 .fillna(0)
 .astype(int))

la_imd_pop_quintile,1.0,2.0,3.0,4.0,5.0
league-group,,,,,
District councils,11,10,39,45,76
Single tier,44,42,42,32,22


The above tables shows the distribution of a *single* league over both district and single tiers.

The alternative approach would be to calculate the distribution for two different sets of quintiles.

In [20]:
final = ldf[["league-group", "la_imd_pop_quintile"]].reset_index()
final = final[~final["la_imd_pop_quintile"].isna()]
final.to_csv(Path("data", "outputs", "imd_joint_ranking.csv"), index=False)

In [21]:
ldf = la.get_la_with_leagues().set_index("local-authority-code")
ldf = ldf.join(df, how="outer")

def calculate_deciles(df):
    # calculate a new score, dividing the summed score by the summed population
    df["la_deprivation_score"] = df["pop_score"] / df["pop"]
    df = df.sort_values("la_deprivation_score", ascending=False)

    df["cum_pop"] = df["pop"].astype("int").cumsum()

    df["la_imd_pop_decile"] = np.ceil(
        df["cum_pop"]/sum(df["pop"]) * 10).astype(int)
    df["la_imd_pop_quintile"] = np.ceil(
        df["cum_pop"]/sum(df["pop"]) * 5).astype(int)

    df = df.drop(columns=["cum_pop"])
    return df

ldf = ldf[ldf["league-group"].isin(["District councils", "Single tier"])]
ldf = ldf.groupby("league-group", as_index=False).apply(calculate_deciles).reset_index()


In [22]:
(ldf
 .reset_index()
 .pivot_table("local-authority-code", index="league-group", columns="la_imd_pop_quintile", aggfunc="count")
 .fillna(0)
 .astype(int)
 .common.row_percentages()
 .style.format(percentage_0dp))


la_imd_pop_quintile,1,2,3,4,5
league-group,,,,,
District councils,20%,20%,20%,19%,20%
Single tier,17%,19%,19%,23%,23%


In [23]:
(ldf
 .reset_index()
 .pivot_table("local-authority-code", index="league-group", columns="la_imd_pop_quintile", aggfunc="count")
 .fillna(0)
 .astype(int))

la_imd_pop_quintile,1,2,3,4,5
league-group,,,,,
District councils,37,37,36,35,36
Single tier,31,34,34,41,42


Undecided on which of these is better. This gives more even buckets, but by knocking the other set of councils (and the people who live there) out of the respective rankings, it brings some councils who wouldn't otherwise be similar directly into comparison. 

Leave this as a question to be resolved later and export two sets of csvs.

In [24]:
final = ldf[["local-authority-code", "league-group", "la_imd_pop_quintile"]]
final = final[~final["la_imd_pop_quintile"].isna()]
final.to_csv(Path("data", "outputs", "imd_split_ranking.csv"), index=False)